<a href="https://colab.research.google.com/github/silverstar0727/pose-estimation/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm==4.56.0 -q
!pip install mediapipe==0.8.3 -q

     |████████████████████████████████| 72 kB 845 kB/s 
     |████████████████████████████████| 67.0 MB 23 kB/s 
     |████████████████████████████████| 14.9 MB 100 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
import csv
import cv2
import numpy as np
import os
import sys
import tqdm

import pandas as pd

from mediapipe.python.solutions import pose as mp_pose

import pickle
from sklearn.externals import joblib
from sklearn.neighbors import KNeighborsClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [6]:
def landmarks(input_frame):
    input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)

      # Initialize fresh pose tracker and run it.
    with mp_pose.Pose(upper_body_only=True) as pose_tracker:
        result = pose_tracker.process(image=input_frame)
        pose_landmarks = result.pose_landmarks
      
      # Save landmarks.
    if pose_landmarks is not None:
        # Check the number of landmarks and take pose landmarks.
        assert len(pose_landmarks.landmark) == 25, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
        pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]

        # Map pose landmarks from [0, 1] range to absolute coordinates to get
        # correct aspect ratio.
        frame_height, frame_width = input_frame.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])

        # Write pose sample to CSV.
        pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.float64).tolist()

        return pose_landmarks

In [7]:
images_in_folder = 'study'
images_out_folder = 'stduy_out'
csv_out_path = 'study_result.csv'

with open(csv_out_path, 'w') as csv_out_file:
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)

    # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_in_folder) if not n.startswith('.')])

    for pose_class_name in pose_class_names:
        print('Bootstrapping ', pose_class_name, file=sys.stderr)

        if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
            os.makedirs(os.path.join(images_out_folder, pose_class_name))

        image_names = sorted([
            n for n in os.listdir(os.path.join(images_in_folder, pose_class_name))
            if not n.startswith('.')])
        for image_name in tqdm.tqdm(image_names, position=0):
            # Load image.
            input_frame = cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name))
            pose_landmarks = landmarks(input_frame)

            try:
                csv_out_writer.writerow([image_name, pose_class_name] + pose_landmarks)
            except:
                pass

Bootstrapping  do
100%|██████████| 183/183 [00:21<00:00,  8.65it/s]
Bootstrapping  do_not
100%|██████████| 162/162 [00:19<00:00,  8.21it/s]


In [8]:
def knn(result_csv_file="study_result.csv"):
    data = pd.read_csv(result_csv_file, header=None)
    x_train, y_train = data.iloc[:, 2:], data.iloc[:, 1]

    classifier = KNeighborsClassifier(n_neighbors = 3)

    classifier.fit(x_train, y_train)
    return classifier

classifier = knn()

In [ ]:
data = pd.read_csv("study_result.csv", header=None)
x, y = data.iloc[:, 2:], data.iloc[:, 1]
correct = 0
for i in range(len(data)):
    sample = np.array(x.iloc[i, :]).reshape(1, -1)
    pred = classifier.predict(sample)
    if pred[0] == y[i]:
        correct += 1
print(correct / len(data))

"""
do = sorted([n for n in os.listdir("study/do_not") if not n.startswith('.')])
do_not = sorted([n for n in os.listdir("study/do_not") if not n.startswith('.')])

for i in do_not:
    input_frame = cv2.imread(f"study/do_not/{i}")
    try:
        result = landmarks(input_frame)
        result = np.array(result).reshape(1, -1)
        print(classifier.predict(result), i)
    except:
        pass
"""

In [ ]:
joblib.dump(classifier, 'model.pkl') 

#clf_from_joblib = joblib.load('model.pkl') 
#clf_from_joblib.predict(x_train)

['model.pkl']